In [2]:
!pip3 install pulp


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [84]:
# from pulp import *
# def calculateOptimalPlan(n, edge_list, supplies, debug=False):
#     assert n >= 1
#     assert all( 0 <= i < n and 0 <= j < n and i != j and c >= 0 for (i,j,c) in edge_list)
#     assert len(supplies) == n
#     # TODO: Formulate the LP for optimal transportation plan and return the solution as a dictionary
#     #       from edges (i,j) to flow from i to j.
#     #       If an edge is not present in the dictionary, we will take its flow to be zero.
# 
#     prob = LpProblem('Optimal_Transportation', LpMinimize)
# 
#     # Make decision variables
#     decision_vars = {}
#     for i, j, _ in edge_list:
#         decision_vars[i,j] = (LpVariable(f"x_{i}_{j}", lowBound=0, cat=LpContinuous))
#     print('Decision Variables:', decision_vars)
# 
#     # Define objective function
#     total_cost = 0
#     for i, j, c in edge_list:
#         total_cost += decision_vars[i, j] * c
#     print('Objective Function:', total_cost)
#     prob += total_cost
# 
#     # Define constraints
#     for v in range(n):
#         in_flow = 0
#         for i, j, _ in edge_list:
#             if j == v:
#                 in_flow += decision_vars[i, j]
# 
#         out_flow = 0
#         for i, j, _ in edge_list:
#             if i == v:
#                 out_flow += decision_vars[i, j]
# 
#         flow_constraint = None
#         if supplies[v] >= 0:
#             flow_constraint = (out_flow - in_flow == abs(supplies[v]))
#         elif supplies[v] < 0: 
#             flow_constraint = (in_flow - out_flow <= abs(supplies[v]))
#         
#         prob += flow_constraint
#     # Solve problem with Pulp
#     print(prob)
#     prob.solve()
#     # 
#     # # Construct the result dictionary
#     result = {}
#     for (i, j) in decision_vars:
#         result[(i, j)] = decision_vars[(i, j)].varValue
# 
#     return result

In [94]:
from pulp import *

def calculateOptimalPlan(n, edge_list, supplies, debug=False):
    assert n >= 1
    assert all(0 <= i < n and 0 <= j < n and i != j and c >= 0 for (i, j, c) in edge_list)
    assert len(supplies) == n
    
    # Create the optimization problem (minimization)
    prob = LpProblem("Transportation_Problem", LpMinimize)
    
    # Create flow variables for each edge
    flows = {}
    edge_costs = {}
    # Create variables and store costs for both directions
    for i, j, c in edge_list:
        flows[(i,j)] = LpVariable(f"flow_{i}_{j}", 0)
        flows[(j,i)] = LpVariable(f"flow_{j}_{i}", 0)
        edge_costs[(i,j)] = c
        edge_costs[(j,i)] = c
    
    # Objective function: minimize total cost
    prob += lpSum(flows[edge] * edge_costs[edge] for edge in flows)
    
    # Flow conservation constraints for each vertex
    for i in range(n):
        # Find all edges connected to vertex i
        outgoing_edges = [(i,j) for i,j in flows.keys() if i == i]
        incoming_edges = [(j,i) for j,i in flows.keys() if i == i]
        
        # Net flow at vertex i must equal its supply/demand
        prob += (lpSum(flows[edge] for edge in outgoing_edges) - 
                lpSum(flows[edge] for edge in incoming_edges) == supplies[i])
    
    # Solve the problem
    status = prob.solve(PULP_CBC_CMD(msg=debug))
    
    if status != 1:  # Not optimal
        return {}
    
    # Create the result dictionary
    result = {}
    for (i,j) in flows:
        flow_value = value(flows[(i,j)])
        if flow_value is not None and flow_value > 0:
            result[(i,j)] = flow_value
            
    return result

In [96]:
def test_solution(n, edge_list, supplies, solution_map, expected_cost):
    cost = 0
    outflows = [0]*n
    inflows = [0]*n
    for (i,j,c) in edge_list:
        if (i,j) in solution_map: 
            flow = solution_map[(i,j)]
            cost += c * flow
            assert flow >= 0, f'flow on edge {(i,j)} is negative --> {flow}'
            outflows[i] += flow 
            inflows[j] += flow
        elif (j,i) in solution_map:
            flow = solution_map[(j,i)]
            cost += c * flow
            assert flow >= 0, f'flow on edge {(j,i)} in negative --> {flow}'
            outflows[j] += flow
            inflows[i] += flow 
    for (i, s) in enumerate(supplies):
        if s > 0:
            assert outflows[i]  - inflows[i] <= s, f'Vertex {i} constraint violated: total outflow = {outflows[i]} inflow = {inflows[i]}, supply = {s}'
        else:
            assert abs(inflows[i]-outflows[i] + s) <= 1E-2,f'Vertex{i} constraint violated: inflow = {inflows[i]} outflow={outflows[i]}, demand = {-s}'
    if expected_cost != None:
        assert abs(expected_cost - cost) <= 1E-02, f'Expected cost: {expected_cost}, your algorithm returned: {cost}'
    print('Test Passed!')

n = 5
edge_list = [
    (0,1, 5), (0, 3, 3), (0, 4, 4),
    (1,2, 9), (1,4, 6),
    (2,3,8),
    (3,4,7)
]
supplies = [-55, 100, -25, 35, -40]
sol_map = calculateOptimalPlan(n, edge_list, supplies, debug=True)
test_solution(n, edge_list, supplies, sol_map,670)


print('5 points!')

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/josiahroa/workspace/cs-masters/5414-greedy-algorithms/karatsuba/.venv/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/3g/sywttvsx0m34y00rmp2v39b00000gn/T/0df5d1cff4fb4b95a86f8f67f673bf4d-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/3g/sywttvsx0m34y00rmp2v39b00000gn/T/0df5d1cff4fb4b95a86f8f67f673bf4d-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 10 COLUMNS
At line 95 RHS
At line 101 BOUNDS
At line 102 ENDATA
Problem MODEL has 5 rows, 14 columns and 0 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Empty problem - 5 rows, 14 columns and 0 elements
Empty problem - 5 rows, 14 columns and 0 elements
Primal infeasible - objective value 0
PrimalInfeasible objective 0 - 0 iterations time 0.002

Result - Linear relaxation infeasible

Enumerated nodes:           0

AssertionError: Vertex0 constraint violated: inflow = 0 outflow=0, demand = 55

In [97]:
# from pulp import *
# 
# def testPath(n, edge_list, supplies, debug=False):
#     # Create a linear programming problem
#     prob = LpProblem("Transportation_Problem", LpMinimize)
#     
#     # Define variables for each edge (i, j) representing flow
#     flow_vars = {(i, j): LpVariable(f'flow_{i}_{j}', lowBound=0, cat='Continuous') for (i, j, _) in edge_list}
#     
#     # Objective function: Minimize the total transportation cost
#     prob += lpSum(flow_vars[i, j] * c for (i, j, c) in edge_list), "Total Cost"
#     
#     # Constraints for each node i
#     for v in range(n):
#         # Net flow = outflow - inflow should be equal to supplies[i]
#         prob += (
#             lpSum(flow_vars[i, j] for (i, j, _) in edge_list if i == i) -
#             lpSum(flow_vars[j, i] for (j, i, _) in edge_list if i == i)
#             == supplies[v], f"Node_{v}_Supply_Demand"
#         )
#     
#     # Solve the problem
#     prob.solve()
#     
#     # Create the solution map (flows for each edge)
#     solution_map = {(i, j): flow_vars[i, j].varValue for (i, j, _) in edge_list if flow_vars[i, j].varValue > 0}
#     
#     if debug:
#         print("Status:", LpStatus[prob.status])
#         print("Optimal Cost:", value(prob.objective))
#         for (i, j), flow in solution_map.items():
#             print(f"Flow from {i} to {j}: {flow}")
#     
#     return solution_map